In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import src.config as config

In [7]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-01-05 12:00:00')


In [8]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]
    
    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)
rides

File 2021-12 was already in local storage
File 2022-01 was already in local storage


,pickup_datetime,pickup_location_id
1017878,2022-12-08 13:42:42,1
1015679,2022-12-08 13:53:15,1
1027426,2022-12-08 14:44:02,1
1023784,2022-12-08 14:54:49,1
1031090,2022-12-08 15:41:02,1
...,...,...
2400784,2023-01-05 11:01:23,265
2400828,2023-01-05 11:01:27,265
2400822,2023-01-05 11:01:48,265
2400777,2023-01-05 11:01:56,265


In [10]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)
ts_data

100%|██████████| 258/258 [00:01<00:00, 230.55it/s]


,pickup_hour,rides,pickup_location_id
0,2022-12-08 12:00:00,3,4
1,2022-12-08 13:00:00,6,4
2,2022-12-08 14:00:00,7,4
3,2022-12-08 15:00:00,10,4
4,2022-12-08 16:00:00,7,4
...,...,...,...
173371,2023-01-05 07:00:00,0,204
173372,2023-01-05 08:00:00,0,204
173373,2023-01-05 09:00:00,0,204
173374,2023-01-05 10:00:00,0,204


In [9]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447


Connected. Call `.close()` to terminate connection gracefully.


In [11]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/173376 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/12447/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1346fe2b0>, None)